# EPA Region 9 — Field Equipment Loadout Sheet Generator

Reads from your AGOL hosted tables and generates a field-ready PDF showing
device specs, sensor detection ranges, target chemicals (with PEL/IDLH), and
cross-sensitivity profiles.

**YOU CAN BREAK THIS FOR EVERYONE** So don't mess with anything. You just need to hit the two arrows that look like this ">>" under the word "Settings" and let it run to the bottom.

**Really, just dont hit Save**

If there is a problem, let Bryson know and he will fix it. Don't worry about warnings. If it works as expected, there isn't a problem.


In [1]:
# ── CELL 1 — Install dependencies ─────────────────────────────────────────────
# Run once per kernel session.

import subprocess
subprocess.run(["pip", "install", "reportlab", "--quiet"], check=True)
print("Dependencies ready.")


DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/tflite_model_maker-0.3.4-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


Dependencies ready.


In [2]:
# ── CELL 2 — Connect and load data ────────────────────────────────────────────

from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd

gis = GIS("home")
print(f"Connected as: {gis.users.me.username}")

# ── Table IDs ──────────────────────────────────────────────────────────────────
TABLE_IDS = {
    "Devices":                   "1af118dda1cc49cbb0150518e8f8f1a8",
    "Device_Full_Profile":       "5bd501c2b5ae4fc8a91c1fd3773a97fe",
    "Chemical_CrossSens_Detail": "e8d175c9e468499dae3f892a4544ee84",
    "Device_Sensor":             "87013439e5c0472fb1bf549fe46c1e3f",  # ← add this
    "Sensors":                   "69bf7ab4c1af46509630a8f93aa656ba",  # ← and this
}

def load_table(table_name):
    """Load a hosted table from AGOL as a pandas DataFrame."""
    item = gis.content.get(TABLE_IDS[table_name])
    layer = item.layers[0] if item.layers else item.tables[0]
    sdf = layer.query(as_df=True)
    drop_cols = [c for c in sdf.columns if c.upper() in
                 ("SHAPE", "OBJECTID", "GLOBALID", "SHAPE__AREA", "SHAPE__LENGTH")]
    return sdf.drop(columns=drop_cols, errors="ignore")

print("Loading tables...")
df_devices = load_table("Devices")
df_profile = load_table("Device_Full_Profile")
df_xsens   = load_table("Chemical_CrossSens_Detail")

print(f"  Devices:                   {len(df_devices)} rows")
print(f"  Device_Full_Profile:       {len(df_profile)} rows")
print(f"  Chemical_CrossSens_Detail: {len(df_xsens)} rows")
print("Done.")


Connected as: Patterson.Bryson_EPAEXT
Loading tables...
  Devices:                   12 rows
  Device_Full_Profile:       157 rows
  Chemical_CrossSens_Detail: 60 rows
Done.


In [3]:
# ── CELL 3 — Column map ────────────────────────────────────────────────────────
# All column name references live here.
# If Cell 2b shows a different name, fix it here — everything downstream updates.
#
# Naming logic from the upload notebook's prefix_cols():
#   Entity tables:   key column stays unprefixed, all others get prefix_
#   Junction tables: all columns stay unprefixed
#
# Columns marked # <- VERIFY depend on your exact Excel column headers.

# Device_Full_Profile — Devices entity (prefix: device_)
COL_DEVICE_ID   = "device_id"
COL_DEVICE_NAME = "device_model"           # <- VERIFY
COL_DEVICE_MFR  = "device_manufacturer"   # <- VERIFY

# Device_Full_Profile — Sensors entity (prefix: sensor_)
COL_SENSOR_ID   = "sensor_id"
COL_SENSOR_NAME = "sensor_plain_name"           # <- VERIFY
COL_SENSOR_TECH = "sensor_technology"     # <- VERIFY
COL_SENSOR_RANGE= "sensor_detection_range"# <- VERIFY (may be _low / _high)
COL_SENSOR_RES  = "sensor_resolution"     # <- VERIFY
COL_SENSOR_RESP = "sensor_response_time"  # <- VERIFY

# Device_Full_Profile — Chemicals entity (prefix: chem_)
COL_CHEM_CAS    = "cas_number"            # key — unprefixed
COL_CHEM_NAME   = "chem_chemical_name"    # <- VERIFY
COL_CHEM_PEL    = "chem_pel_ppm"              # <- VERIFY
COL_CHEM_IDLH   = "chem_idlh_ppm"             # <- VERIFY

# Device_Full_Profile — Sensor_Chemical junction (unprefixed)
COL_REL_TYPE    = "relationship_type"     # 'primary' or 'detectable'
COL_CORRECTION  = "correction_factor"     # <- VERIFY (may not exist yet)
COL_CHEM_ID     = "chemical_id"           # links to cas_number

# Chemical_CrossSens_Detail
COL_INTERF_NAME = "chem_chemical_name"    # interferent chemical name <- VERIFY
COL_XSENS_SUM   = "cross_sens_summary"    # pre-computed summary string
COL_RESP_TYPE   = "response_type"
COL_NOTES       = "notes"

def safe(val, fallback="—"):
    """Return string value or fallback for NaN/None/empty."""
    if val is None:
        return fallback
    try:
        if pd.isna(val):
            return fallback
    except (TypeError, ValueError):
        pass
    s = str(val).strip()
    return s if s else fallback

print("Column map loaded.")


Column map loaded.


In [4]:
# ── CELL 4 — PDF builder ───────────────────────────────────────────────────────

import io
from datetime import datetime
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle,
    HRFlowable, KeepTogether
)
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT

EPA_BLUE   = colors.HexColor("#1a3a5c")
EPA_MID    = colors.HexColor("#4a7fb5")
EPA_LIGHT  = colors.HexColor("#e8f0f7")
WARN_RED   = colors.HexColor("#c0392b")
WARN_AMBER = colors.HexColor("#e67e22")
POISON_BG  = colors.HexColor("#fdecea")
OK_GREEN   = colors.HexColor("#1e7e34")
LIGHT_GRAY = colors.HexColor("#f5f5f5")
MID_GRAY   = colors.HexColor("#cccccc")
DARK_GRAY  = colors.HexColor("#444444")
WHITE      = colors.white

def S(name, **kw):
    return ParagraphStyle(name, **kw)

def flag_color(summary, notes=""):
    combined = (summary + " " + notes).upper()
    if "POISON" in combined or "PERMANENT" in combined:
        return WARN_RED, WHITE
    if "NO RESPONSE" in combined:
        return WARN_RED, WHITE
    if "OVER-READ" in combined or "HIGH INTERFERENCE" in combined:
        return WARN_AMBER, WHITE
    return LIGHT_GRAY, DARK_GRAY

def tbl(data, widths, style_cmds):
    t = Table(data, colWidths=widths)
    t.setStyle(TableStyle(style_cmds))
    return t

def pad(top=5, bot=5, left=8, right=8):
    return [
        ("TOPPADDING",    (0,0), (-1,-1), top),
        ("BOTTOMPADDING", (0,0), (-1,-1), bot),
        ("LEFTPADDING",   (0,0), (-1,-1), left),
        ("RIGHTPADDING",  (0,0), (-1,-1), right),
    ]

def build_pdf(selected_device_id, selected_sensor_ids, site, incident_no, generated_by):
    buffer = io.BytesIO()
    W = letter[0] - 1.2 * inch

    doc = SimpleDocTemplate(
        buffer, pagesize=letter,
        leftMargin=0.6*inch, rightMargin=0.6*inch,
        topMargin=0.6*inch,  bottomMargin=0.6*inch
    )

    profile = df_profile[
        (df_profile[COL_DEVICE_ID] == selected_device_id) &
        (df_profile[COL_SENSOR_ID].isin(selected_sensor_ids))
    ].copy()

    xsens = (
    df_xsens[df_xsens[COL_SENSOR_ID].isin(selected_sensor_ids)]
    .drop_duplicates(subset=["sensor_id", "chemical_id"])
    .copy()
    )
    
    dev_row = profile.iloc[0] if len(profile) > 0 else {}

    story = []

    # Header
    title_s = S("T",  fontSize=16, fontName="Helvetica-Bold", textColor=WHITE, leading=20)
    sub_s   = S("Su", fontSize=9,  fontName="Helvetica",
                textColor=colors.HexColor("#b0c8e0"), leading=12)
    hdr = tbl([[
        Paragraph("EPA EMERGENCY RESPONSE", title_s),
        Paragraph("Field Equipment Loadout Sheet", sub_s),
    ]], [W*0.55, W*0.45], [
        ("BACKGROUND",    (0,0), (-1,-1), EPA_BLUE),
        ("VALIGN",        (0,0), (-1,-1), "MIDDLE"),
        ("LEFTPADDING",   (0,0), (0,-1),  16),
        ("RIGHTPADDING",  (-1,0),(-1,-1), 16),
    ] + pad(14,14,16,16))
    story += [hdr, Spacer(1, 8)]

    # Meta bar
    ml = S("ML", fontSize=7.5, fontName="Helvetica-Bold", textColor=EPA_MID, leading=11)
    mv = S("MV", fontSize=9,   fontName="Helvetica",      textColor=DARK_GRAY, leading=11)
    meta = tbl([
        [Paragraph("SITE / INCIDENT", ml), Paragraph("INCIDENT NO.", ml),
         Paragraph("GENERATED BY", ml),    Paragraph("DATE", ml)],
        [Paragraph(site, mv),              Paragraph(incident_no, mv),
         Paragraph(generated_by, mv),
         Paragraph(datetime.now().strftime("%B %d, %Y"), mv)],
    ], [W*0.38, W*0.18, W*0.22, W*0.22], [
        ("BACKGROUND", (0,0), (-1,-1), EPA_LIGHT),
        ("LINEBELOW",  (0,0), (-1,0),  0.5, EPA_MID),
        ("BOX",        (0,0), (-1,-1), 0.5, MID_GRAY),
    ] + pad(6,6,10,10))
    story += [meta, Spacer(1, 12)]

    # Device banner
    dh = S("DH", fontSize=13, fontName="Helvetica-Bold", textColor=WHITE, leading=16)
    ds = S("DS", fontSize=8.5, fontName="Helvetica",
           textColor=colors.HexColor("#c8ddf0"), leading=11)
    device_banner = tbl([[
        Paragraph(safe(dev_row.get(COL_DEVICE_NAME), selected_device_id), dh),
        Paragraph(
            f"{safe(dev_row.get(COL_DEVICE_MFR))}  ·  {selected_device_id}  ·  "
            f"{len(selected_sensor_ids)} sensor(s) selected", ds),
    ]], [W*0.45, W*0.55], [
        ("BACKGROUND",    (0,0), (-1,-1), EPA_BLUE),
        ("VALIGN",        (0,0), (-1,-1), "MIDDLE"),
        ("LEFTPADDING",   (0,0), (0,-1),  14),
    ] + pad(10,10,14,14))
    story += [device_banner, Spacer(1, 10)]

    # Sensors
    lbl = S("LB", fontSize=7.5, fontName="Helvetica-Bold", textColor=EPA_MID, leading=10)
    val = S("VL", fontSize=8.5, fontName="Helvetica",      textColor=DARK_GRAY, leading=11)
    th  = S("TH", fontSize=7.5, fontName="Helvetica-Bold", textColor=WHITE,
            alignment=TA_CENTER, leading=10)
    thl = S("THL",fontSize=7.5, fontName="Helvetica-Bold", textColor=WHITE, leading=10)

    for sensor_id in selected_sensor_ids:
        sensor_rows = profile[profile[COL_SENSOR_ID] == sensor_id]
        if sensor_rows.empty:
            continue
        s0 = sensor_rows.iloc[0]
        blocks = []

        # Sensor name + ID
        sn_s  = S("SN",  fontSize=11, fontName="Helvetica-Bold", textColor=EPA_BLUE, leading=14)
        sid_s = S("SID", fontSize=8,  fontName="Helvetica",      textColor=colors.gray, leading=10)
        sname = tbl([[
            Paragraph(safe(s0.get(COL_SENSOR_NAME), sensor_id), sn_s),
            Paragraph(sensor_id, sid_s),
        ]], [W*0.72, W*0.28], [
            ("VALIGN",        (0,0), (-1,-1), "BOTTOM"),
            ("LEFTPADDING",   (0,0), (-1,-1), 0),
            ("RIGHTPADDING",  (0,0), (-1,-1), 0),
            ("TOPPADDING",    (0,0), (-1,-1), 0),
            ("BOTTOMPADDING", (0,0), (-1,-1), 3),
        ])
        blocks.append(sname)

        # Specs bar
        specs = tbl([
            [Paragraph("TECHNOLOGY", lbl), Paragraph("DETECTION RANGE", lbl),
             Paragraph("RESOLUTION", lbl), Paragraph("RESPONSE TIME", lbl),
             Paragraph("PART NUMBER", lbl)],
            [Paragraph(safe(s0.get(COL_SENSOR_TECH)),          val),
             Paragraph(safe(s0.get(COL_SENSOR_RANGE)),          val),
             Paragraph(safe(s0.get(COL_SENSOR_RES)),            val),
             Paragraph(safe(s0.get(COL_SENSOR_RESP)),           val),
             Paragraph(safe(s0.get("sensor_part_number")),      val)],
        ], [W*0.22, W*0.24, W*0.16, W*0.18, W*0.20], [
            ("BACKGROUND", (0,0), (-1,-1), LIGHT_GRAY),
            ("LINEBELOW",  (0,0), (-1,0),  0.5, MID_GRAY),
            ("BOX",        (0,0), (-1,-1), 0.5, MID_GRAY),
        ] + pad(5,5,8,8))
        blocks += [specs, Spacer(1, 6)]

        # Detected chemicals — primary then detectable
        cn_s  = S("CN",  fontSize=8.5, fontName="Helvetica-Bold", textColor=DARK_GRAY, leading=11)
        cas_s = S("CAS", fontSize=7,   fontName="Helvetica",      textColor=colors.gray, leading=9)
        cv_s  = S("CV",  fontSize=8.5, fontName="Helvetica",
                  textColor=DARK_GRAY, alignment=TA_CENTER, leading=11)

        for section_label, section_rows, header_bg in [
            ("PRIMARY TARGET CHEMICALS", sensor_rows[sensor_rows[COL_REL_TYPE]=="primary"],   EPA_BLUE),
            ("ALSO DETECTABLE",          sensor_rows[sensor_rows[COL_REL_TYPE]=="detectable"], EPA_MID),
        ]:
            if section_rows.empty:
                continue
            chem_hdr = tbl([[
                Paragraph(section_label, thl),
                Paragraph("CORRECTION FACTOR", th),
                Paragraph("PEL", th),
                Paragraph("IDLH", th),
            ]], [W*0.46, W*0.18, W*0.18, W*0.18], [
                ("BACKGROUND", (0,0), (-1,-1), header_bg),
                ("VALIGN",     (0,0), (-1,-1), "MIDDLE"),
            ] + pad(5,5,8,8))
            blocks.append(chem_hdr)

            for i, (_, crow) in enumerate(section_rows.iterrows()):
                row_bg = WHITE if i % 2 == 0 else LIGHT_GRAY
                cf = safe(crow.get(COL_CORRECTION), "")
                cf_display = f"{cf}x" if cf and cf != "\u2014" else "1.0x (direct)"
                chem_cell  = Paragraph(safe(crow.get(COL_CHEM_NAME)), cn_s)
                cas_cell   = Paragraph(f"CAS: {safe(crow.get(COL_CHEM_CAS))}", cas_s)
                inner = Table([[chem_cell],[cas_cell]], colWidths=[W*0.46],
                              style=TableStyle([
                                  ("LEFTPADDING",   (0,0),(-1,-1), 0),
                                  ("RIGHTPADDING",  (0,0),(-1,-1), 0),
                                  ("TOPPADDING",    (0,0),(-1,-1), 1),
                                  ("BOTTOMPADDING", (0,0),(-1,-1), 1),
                              ]))
                chem_row = tbl([[
                    inner,
                    Paragraph(cf_display,                    cv_s),
                    Paragraph(safe(crow.get(COL_CHEM_PEL)),  cv_s),
                    Paragraph(safe(crow.get(COL_CHEM_IDLH)), cv_s),
                ]], [W*0.46, W*0.18, W*0.18, W*0.18], [
                    ("BACKGROUND", (0,0), (-1,-1), row_bg),
                    ("LINEBELOW",  (0,0), (-1,-1), 0.3, MID_GRAY),
                    ("VALIGN",     (0,0), (-1,-1), "MIDDLE"),
                ] + pad(4,4,8,8))
                blocks.append(chem_row)

        blocks.append(Spacer(1, 8))

        # Cross-sensitivities
        sensor_xsens = xsens[xsens[COL_SENSOR_ID] == sensor_id]
        if not sensor_xsens.empty:
            xs_hdr = tbl([[
                Paragraph("INTERFERENT / CROSS-SENSITIVITY", thl),
                Paragraph("RESPONSE", th),
            ]], [W*0.38, W*0.62], [
                ("BACKGROUND", (0,0), (-1,-1), colors.HexColor("#2c2c2c")),
            ] + pad(5,5,8,8))
            blocks.append(xs_hdr)

            xn_s  = S("XN",  fontSize=8.5, fontName="Helvetica-Bold", textColor=DARK_GRAY, leading=11)
            xnt_s = S("XNT", fontSize=7.5, fontName="Helvetica-Oblique", textColor=colors.gray, leading=9)
            xs_s  = S("XS",  fontSize=8.5, fontName="Helvetica",      textColor=DARK_GRAY, leading=11)
            xsw_s = S("XSW", fontSize=8.5, fontName="Helvetica-Bold", textColor=WARN_RED,  leading=11)

            for i, (_, xrow) in enumerate(sensor_xsens.iterrows()):
                summary  = safe(xrow.get(COL_XSENS_SUM))
                notes    = safe(xrow.get(COL_NOTES), "")
                is_poison= "POISON" in (summary + notes).upper()
                row_bg   = POISON_BG if is_poison else (WHITE if i % 2 == 0 else LIGHT_GRAY)
                interf   = safe(xrow.get(COL_INTERF_NAME))
                rt       = safe(xrow.get(COL_RESP_TYPE), "")
                interf_cell = Table([
                    [Paragraph(interf, xn_s)],
                    [Paragraph(rt, xnt_s)],
                ], colWidths=[W*0.38], style=TableStyle([
                    ("LEFTPADDING",   (0,0),(-1,-1), 0),
                    ("RIGHTPADDING",  (0,0),(-1,-1), 0),
                    ("TOPPADDING",    (0,0),(-1,-1), 1),
                    ("BOTTOMPADDING", (0,0),(-1,-1), 1),
                ]))
                xs_row = tbl([[
                    interf_cell,
                    Paragraph(summary, xsw_s if is_poison else xs_s),
                ]], [W*0.38, W*0.62], [
                    ("BACKGROUND", (0,0), (-1,-1), row_bg),
                    ("LINEBELOW",  (0,0), (-1,-1), 0.3, MID_GRAY),
                    ("VALIGN",     (0,0), (-1,-1), "MIDDLE"),
                ] + pad(5,5,8,8))
                blocks.append(xs_row)

        blocks.append(Spacer(1, 14))
        story.append(KeepTogether(blocks))
        story.append(HRFlowable(width=W, thickness=0.5, color=MID_GRAY))
        story.append(Spacer(1, 10))

    # Footer
    ft = S("FT", fontSize=7, fontName="Helvetica", textColor=colors.gray,
           alignment=TA_CENTER, leading=9)
    story += [
        Spacer(1, 4),
        Paragraph(
            "PEL = OSHA Permissible Exposure Limit (8-hr TWA).  "
            "IDLH = Immediately Dangerous to Life or Health.  "
            "Cross-sensitivity data from manufacturer specifications.  "
            "Verify instrument calibration before deployment.", ft),
        Spacer(1, 4),
        Paragraph(
            f"Generated: {datetime.now().strftime('%B %d, %Y %H:%M')}  ·  "
            "EPA Region 9 Emergency Response  ·  "
            "Source: Air Monitoring Reference DB  ·  DRAFT — NOT FOR REGULATORY USE", ft),
    ]

    doc.build(story)
    buffer.seek(0)
    return buffer.read()

print("PDF builder loaded.")


PDF builder loaded.


In [5]:
# ── CELL 5 — Interactive UI ────────────────────────────────────────────────────
# Run this cell to display the loadout generator.
# Re-run after updating data to refresh the device list.

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import base64

# Build device dropdown from Device_Full_Profile
device_options = (
    df_profile[[COL_DEVICE_ID, COL_DEVICE_NAME]]
    .drop_duplicates(subset=[COL_DEVICE_ID])
    .sort_values(COL_DEVICE_NAME)
)
device_dropdown_options = [("— Select a device —", None)] + [
    (f"{safe(row[COL_DEVICE_NAME])}  ({row[COL_DEVICE_ID]})", row[COL_DEVICE_ID])
    for _, row in device_options.iterrows()
]

# Widgets
header_html = widgets.HTML("""
    <div style="font-family:sans-serif; padding:12px 0 4px 0;">
        <h3 style="color:#1a3a5c; margin:0 0 4px 0;">
            EPA Region 9 — Field Equipment Loadout Generator
        </h3>
        <p style="color:#666; font-size:13px; margin:0;">
            Select a device, confirm which sensors are installed,
            fill in site info, and generate a field reference PDF.
        </p>
    </div>
""")

site_input = widgets.Text(
    placeholder="e.g. Richmond Chevron Refinery — Emergency Response",
    description="Site:",
    layout=widgets.Layout(width="520px"),
    style={"description_width": "80px"})

incident_input = widgets.Text(
    placeholder="e.g. ER-2026-0214",
    description="Incident #:",
    layout=widgets.Layout(width="300px"),
    style={"description_width": "80px"})

user_input = widgets.Text(
    placeholder="e.g. B. McFarland",
    description="Your name:",
    layout=widgets.Layout(width="300px"),
    style={"description_width": "80px"})

device_dropdown = widgets.Dropdown(
    options=device_dropdown_options,
    value=None,
    description="Device:",
    layout=widgets.Layout(width="480px"),
    style={"description_width": "80px"})

sensor_header = widgets.HTML("")
sensor_box    = widgets.VBox([])

generate_btn = widgets.Button(
    description="Generate Loadout PDF",
    button_style="primary",
    icon="file-pdf-o",
    layout=widgets.Layout(width="220px", height="40px"),
    disabled=True)

status_label = widgets.Label("Select a device to begin.")
download_area = widgets.Output()

def on_device_change(change):
    sensor_box.children = []
    sensor_header.value = ""
    generate_btn.disabled = True
    download_area.clear_output()
    device_id = change["new"]
    if not device_id:
        status_label.value = "Select a device to begin."
        return

    linked = (
        df_profile[df_profile[COL_DEVICE_ID] == device_id]
        [[COL_SENSOR_ID, COL_SENSOR_NAME, "sensor_part_number"]]
        .drop_duplicates(subset=[COL_SENSOR_ID])
        .sort_values(COL_SENSOR_NAME)
    )
    if linked.empty:
        status_label.value = f"No sensors found for {device_id}."
        return

    checkboxes = []
    for _, row in linked.iterrows():
        pn = str(row['sensor_part_number']) if pd.notna(row['sensor_part_number']) else ""
        pn_str = f"  ·  {pn}" if pn else ""
        cb = widgets.Checkbox(
            value=False,
            description=f"{safe(row[COL_SENSOR_NAME])}{pn_str}  ({row[COL_SENSOR_ID]})",
            indent=False,
            layout=widgets.Layout(width="480px"))
        cb.sensor_id = row[COL_SENSOR_ID]
        checkboxes.append(cb)

    sensor_header.value = (
        f"<b style='font-family:sans-serif; color:#1a3a5c;'>"
        f"Sensors linked to this device ({len(checkboxes)} found — "
        f"uncheck any not physically installed):</b>")
    sensor_box.children = checkboxes
    generate_btn.disabled = False
    status_label.value = f"{len(checkboxes)} sensor(s) available."

device_dropdown.observe(on_device_change, names="value")

def on_generate_clicked(b):
    download_area.clear_output()
    device_id    = device_dropdown.value
    sensor_ids   = [cb.sensor_id for cb in sensor_box.children if cb.value]
    site         = site_input.value.strip()
    incident_no  = incident_input.value.strip() or "—"
    generated_by = user_input.value.strip()

    if not device_id:
        status_label.value = "⚠  Select a device first."; return
    if not sensor_ids:
        status_label.value = "⚠  Select at least one sensor."; return
    if not site:
        status_label.value = "⚠  Enter a site / incident name."; return
    if not generated_by:
        status_label.value = "⚠  Enter your name."; return

    status_label.value = "Building PDF..."
    generate_btn.disabled = True
    try:
        pdf_bytes = build_pdf(
            selected_device_id=device_id,
            selected_sensor_ids=sensor_ids,
            site=site, incident_no=incident_no, generated_by=generated_by)

        b64      = base64.b64encode(pdf_bytes).decode()
        inc_slug = incident_no.replace("/","-").replace(" ","_")
        dev_slug = device_id.replace("/","-")
        filename = f"Loadout_{inc_slug}_{dev_slug}_{datetime.now().strftime('%Y%m%d')}.pdf"
        with download_area:
            display(HTML(f"""
            <a href="data:application/pdf;base64,{b64}" download="{filename}"
               style="display:inline-block; margin-top:10px; padding:10px 22px;
                      background:#1a3a5c; color:white; font-family:sans-serif;
                      font-size:14px; border-radius:4px; text-decoration:none;">
                &#8595; Download {filename}
            </a>
            <span style="font-family:sans-serif; font-size:12px; color:#666; margin-left:12px;">
                {len(sensor_ids)} sensor(s) &middot; {len(pdf_bytes)//1024} KB
            </span>
            """))
        status_label.value = f"✓ Ready — {len(sensor_ids)} sensor(s), {incident_no}."
    except Exception as e:
        status_label.value = f"Error: {e}"
        raise
    finally:
        generate_btn.disabled = False

generate_btn.on_click(on_generate_clicked)

display(widgets.VBox([
    header_html,
    widgets.HTML("<hr style='border:none;border-top:1px solid #ddd;margin:8px 0;'>"),
    widgets.HTML("<b style='font-family:sans-serif;'>Incident Info:</b>"),
    site_input,
    widgets.HBox([incident_input, widgets.HTML("&nbsp;&nbsp;&nbsp;"), user_input]),
    widgets.HTML("<br>"),
    widgets.HTML("<b style='font-family:sans-serif;'>Device:</b>"),
    device_dropdown,
    widgets.HTML("<br>"),
    sensor_header,
    sensor_box,
    widgets.HTML("<br>"),
    generate_btn,
    status_label,
    download_area,
], layout=widgets.Layout(padding="20px", max_width="700px")))